In [1]:
import pandas as pd
month_cols = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [2]:
sub_lgb = pd.read_csv("../submissions/lgb_baseline_clean_post_smo.csv")
sub_const = pd.read_csv("../submissions/mean_1112_pred_post.csv")
sub_lgb = sub_lgb.sort_values(["meter_id"]).reset_index(drop=True)
sub_const = sub_const.sort_values(["meter_id"]).reset_index(drop=True)

print(sub_lgb.shape)
print(sub_const.shape)

(3248, 13)
(3248, 13)


In [3]:
sub = sub_lgb.copy()
sub[month_cols] = 0.8 * sub_lgb[month_cols] + 0.2 * sub_const[month_cols] 

print(sub.shape)
sub[month_cols] = sub[month_cols].values.round()
# sub.to_csv("../submissions/lgb_mean_ensemble.csv", index=False, header=True)
sub.head()

(3248, 13)


,meter_id,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,105.0,100.0,100.0,103.0,99.0,106.0,102.0,107.0,100.0,97.0,90.0,90.0
1,0x000f697092696c27b0bb489ea4c11280ef72ab82,583.0,548.0,546.0,520.0,459.0,472.0,462.0,475.0,470.0,469.0,543.0,497.0
2,0x0019979ee12c59accd24d1c83291528ced1bb734,205.0,196.0,198.0,193.0,174.0,187.0,183.0,186.0,178.0,174.0,194.0,183.0
3,0x001d6227832325ab6167b82d6d7175d254f57c3a,141.0,135.0,135.0,125.0,114.0,120.0,121.0,123.0,121.0,117.0,135.0,124.0
4,0x005958406351bb29580475df698b5f1070096397,333.0,322.0,320.0,298.0,272.0,282.0,281.0,284.0,278.0,276.0,320.0,294.0


In [4]:
# smooth
sub_copy = sub.copy()

df = pd.read_pickle("../input/consumption_month.pkl")
sub = sub.merge(df, on="meter_id")

sub_copy["Jan"] = (sub["2017-12"] + sub["Jan"] + sub["Feb"]) / 3
sub_copy["Feb"] = (sub["Jan"] + sub["Feb"] + sub["Mar"]) / 3
sub_copy["Mar"] = (sub["Feb"] + sub["Mar"] + sub["Apr"]) / 3
sub_copy["Apr"] = (sub["Mar"] + sub["Apr"] + sub["Jun"]) / 3
sub_copy["May"] = (sub["Apr"] + sub["May"] + sub["Jun"]) / 3
sub_copy["Jun"] = (sub["May"] + sub["Jun"] + sub["Jul"]) / 3
sub_copy["Jul"] = (sub["Jun"] + sub["Jul"] + sub["Aug"]) / 3
sub_copy["Aug"] = (sub["Jul"] + sub["Aug"] + sub["Sep"]) / 3
sub_copy["Sep"] = (sub["Aug"] + sub["Sep"] + sub["Oct"]) / 3
sub_copy["Oct"] = (sub["Sep"] + sub["Oct"] + sub["Nov"]) / 3
sub_copy["Nov"] = (sub["Oct"] + sub["Nov"] + sub["Dec"]) / 3
sub_copy["Dec"] = (sub["Nov"] + sub["Dec"]) / 2

sub_copy[month_cols] = sub_copy[month_cols].values.round()
# sub_copy.to_csv("../submissions/lgb_mean_ensemble_smo.csv", index=False, header=True)

In [5]:
sub_scale = sub_copy.copy()

# up-scaling for summer season and down-scaling for winter season
scaler = [1.1, 1.05, 1.1, 1.05, 1.0, 1.0, 1.0, 0.75, 1, 0.95, 1.15, 1.15]
for idx, col in enumerate(month_cols):
    sub_scale.loc[:, col] *= scaler[idx]

# sub_scale.to_csv(f"../submissions/lgb_baseline_clean_post_smo_scale.csv", index=False, header=True)